# Importing data

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Readind the data
X_full = pd.read_csv('Data/train.csv', index_col='Id')
X_test_full = pd.read_csv('Data/test.csv', index_col='Id')

# Removing rows with missing target
X_full.dropna(axis=0, subset = ['SalePrice'], inplace = True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis = 1, inplace = True)

# Kepping only numerical predictions
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Splitting data for training and validation data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)


In [2]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


# Dealing with missing values

In [3]:
#Checking how many missing values occur in dataset

print("Dataset dimnensions: {}".format(X_train.shape))

counting_missing_values_in_col = X_train.isnull().sum()
print(counting_missing_values_in_col[counting_missing_values_in_col > 0])

Dataset dimnensions: (1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


The output shows us that in this case there are not so many missing values, so beside LotFrontage we could drop them, but in this notebook I will try different aproaches with dealing with missing data for training purpose. 
<br>
To have a comparision how specific ways of dealing with missing data change an accurcy of model we firstly should build a model with no changes in data

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)
#print((score_dataset(X_train, X_valid, y_train, y_valid)))

## Dropping down rows with missing data

In [5]:
columns_with_nan = [col for col in X_train.columns if X_train[col].isna().any()]

reduced_X_train = X_train.drop(columns_with_nan, axis= 1)
reduced_X_valid = X_valid.drop(columns_with_nan, axis = 1)

In [6]:
print("MAE for model with dropped nan columns: \n{}".format(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid)))

MAE for model with dropped nan columns: 
17837.82570776256


## Imputation

In [7]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train)) 
imputed_X_valid = pd.DataFrame(imputer.fit_transform(X_valid))

#imputation removes column names so the need to put them back occurs
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [8]:
print("MAE for model with imputed nan values: \n{}".format(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid)))

MAE for model with imputed nan values: 
18056.85163242009


We can see, that imputing missing values casuses our model to pefrorm slithly worse than dropping columns. 
<br>
While this can probably partially be attributed to noise in the dataset, another potential explanation is that the imputation method is not a great match to this dataset. That is, maybe instead of filling in the mean value, it makes more sense to set every missing value to a value of 0, to fill in the most frequently encountered value, or to use some other method. For instance, consider the GarageYrBlt column (which indicates the year that the garage was built). It's likely that in some cases, a missing value could indicate a house that does not have a garage.
<br>
We can also check if others imputing strategies bring us different results

In [9]:
median_imputer = SimpleImputer(strategy='median')

median_imputed_X_train = pd.DataFrame(median_imputer.fit_transform(X_train)) 
median_imputed_X_valid = pd.DataFrame(median_imputer.fit_transform(X_valid))

#imputation removes column names so the need to put them back occurs
median_imputed_X_train.columns = X_train.columns
median_imputed_X_valid.columns = X_valid.columns

In [10]:
print("MAE for modael with imputed nan values: \n{}".format(score_dataset(median_imputed_X_train, median_imputed_X_valid, y_train, y_valid)))

MAE for modael with imputed nan values: 
17786.276735159816


# Categorical variables

To keep this simple we will use data with dropped missing values


In [11]:
na_cols = [col for col in X_full.columns if X_full[col].isna().any()]
X_cat = X_full.drop(na_cols, axis = 1)
X_cat_train, X_cat_valid, y_cat_train, y_cat_valid = train_test_split(X_cat, y, train_size =0.8, test_size = 0.2, random_state = 0)

We alredy have MAE calculated for dropping categorical data values

In [12]:
print("MAE for model with dropped categorical columns: \n{}".format(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid)))

MAE for model with dropped categorical columns: 
17837.82570776256


## Label Encoding

First we must make sure that all values in categorical data in colums that exist in test data exist in training data as well

In [13]:
categorical_col = [col for col in X_cat_train.columns if X_cat_train[col].dtype == 'object']
good_col = [col for col in categorical_col if set(X_cat_train[col]) == set(X_cat_valid[col])]
bad_col = list(set(categorical_col) - set(good_col))

In [14]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

label_X_train = X_cat_train.drop(bad_col, axis = 1)
label_X_valid = X_cat_valid.drop(bad_col, axis = 1)

for col in good_col:
    label_X_train[col] = encoder.fit_transform(label_X_train[col])
    label_X_valid[col] = encoder.transform(label_X_valid[col])

In [15]:
print("MAE for model with label encoding: \n{}".format(score_dataset(label_X_train, label_X_valid, y_cat_train, y_cat_valid)))

MAE for model with label encoding: 
17575.291883561644


## One-hot Encoding

One-hot encoding creates new columns indicating the presence (or absence) of each possible value in the original data. To avoid creating to many new columns, lets see how many unique values occurs in each categorical column


In [16]:
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: X_cat_train[col].nunique(), categorical_col))
d = dict(zip(categorical_col, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

[('Street', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('PavedDrive', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('ExterQual', 4),
 ('KitchenQual', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Condition2', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('Heating', 6),
 ('Functional', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

In [17]:
col_to_hot_encode = [col for col in categorical_col if X_cat_train[col].nunique() < 10]

In [18]:
from sklearn.preprocessing import OneHotEncoder

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse = False)
OH_col_train = pd.DataFrame(OH_encoder.fit_transform(X_cat_train[col_to_hot_encode]))
OH_col_valid = pd.DataFrame(OH_encoder.transform(X_cat_valid[col_to_hot_encode]))

#OH removed index so we need to put it back
OH_col_train.index = X_cat_train.index
OH_col_valid.index = X_cat_valid.index

#dropping down categorical columns to insert enocoded ones
num_X_train = X_cat_train.drop(categorical_col, axis = 1)
num_X_valid = X_cat_valid.drop(categorical_col, axis = 1)

#joing with OH colums
OH_X_train = pd.concat([num_X_train, OH_col_train], axis = 1)
OH_X_valid = pd.concat([num_X_valid, OH_col_valid], axis = 1)

In [19]:
print("MAE for model with One-Hot encoding: \n{}".format(score_dataset(OH_X_train, OH_X_valid, y_cat_train, y_cat_valid)))

MAE for model with One-Hot encoding: 
17525.345719178084
